# 2.2.4.	Search - Depth to Source#

Here we apply the tilt-depth method discussed in Notebook 2.1.4. to estimate depth to source in the Search Phase II magnetic data. Depth can be accurately derived from the tilt angle where contacts between magnetically contrasted geological units are vertical. Because of the typically complex nature of true geological contacts, calculated depths calculated from real magnetic data sets are usually approximations. The results should always be considered in light of expected geology, and ideally should be interpreted alongside geology, topographic data, and other geophysical data and models if these data are available.

<!-- <img src="./images/SearchQuestII.png"> -->


## Total magnetic intensity and area selection

Below is the gridded total magnetic intensity data from the Search Phase II project. The magnetic data is interpolated onto a 50 $m^2$ grid, plotted using coordinate system NAD 83 UTM Zone 9.  

A localized area or feature of interest can be windowed out for investigation using the tilt-depth method. The data 'window' shown in the left map (black outline) can be resized and moved anywhere over the data using the slider bars.


In [1]:
## Load all libraries and setup the synthetic problem
from library.Mag import Mag, ProblemSetter, MathUtils, Simulator, DataIO
import numpy as np

# Load data grid
workDir = './assets/Search/'
fName = 'MAG_UTM09.tiff'

dataGrid = DataIO.loadGeoTiffFile(workDir + fName, plotIt=False)

In [2]:
selection = ProblemSetter.setDataExtentWidget(dataGrid)
display(selection)

interactive(children=(FloatSlider(value=669500.0, continuous_update=False, description='East', max=811100.0, min=578550.0, step=500.0), FloatSlider(value=6069500.0, continuous_update=False, description='North', max=6150300.0, min=5901050.0, step=10.0), FloatSlider(value=30000.0, continuous_update=False, description='Width', max=100000.0, min=1000.0, step=1000.0), FloatSlider(value=30000.0, continuous_update=False, description='Height', max=100000.0, min=1000.0, step=1000.0), Output()), _dom_classes=('widget-interact',))

## Option to upward continue the grid

In [3]:
selection = Simulator.gridFiltersWidget(selection.result, gridFilter='upwardContinuation')
display(selection)

interactive(children=(FloatSlider(value=90.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=15.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.9, continuous_update=False, description='ColorTransp', max=1.0, step=0.05), FloatSlider(value=0.5, continuous_update=False, description='HSTransp', max=1.0, step=0.05), FloatSlider(value=5.0, continuous_update=False, description='vScale', max=10.0, min=1.0, step=1.0), Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), Dropdown(description='Grid Filters', options=('upwardContinuation', 'TMI'), value='upwardContinuation'), FloatSlider(value=0.0, continuous_update=False, description='UpwardDistance', max=500.0, step=10.0), ToggleButton(value=False, description='Refresh', icon='check', tooltip='Description'), ToggleButton(value=False, description='Export Grid', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))

## Tilt angle


In [4]:
polylines, attributes = MathUtils.estimateDepth(selection.result)

# Export to shapefile
DataIO.exportShapefile(polylines, attributes, EPSGCode=26909, fileName='Tilt2Depth', label='AverageDepth')

## Tilt-depth

In [5]:
scatterData = {}
scatterData['x'] = np.vstack(polylines)[:,0]
scatterData['y'] = np.vstack(polylines)[:,1]
scatterData['size'] = 1
scatterData['c'] = np.concatenate(attributes)
scatterData['cmap'] = 'viridis_r'
scatterData['clim'] = [0, 1000]

# view = Simulator.dataHillsideWidget(selection.result,figName='Tilt2Depth', scatterData=scatterData)
param = Simulator.gridFiltersWidget(selection.result, gridFilter='tiltAngle', EPSGCode=26909, scatterData=scatterData)
display(param)

interactive(children=(FloatSlider(value=90.0, continuous_update=False, description='SunAzimuth', max=360.0, step=5.0), FloatSlider(value=15.0, continuous_update=False, description='SunAngle', max=90.0, step=5.0), FloatSlider(value=0.9, continuous_update=False, description='ColorTransp', max=1.0, step=0.05), FloatSlider(value=0.5, continuous_update=False, description='HSTransp', max=1.0, step=0.05), FloatSlider(value=5.0, continuous_update=False, description='vScale', max=10.0, min=1.0, step=1.0), Dropdown(description='ColorMap', index=3, options=('viridis', 'plasma', 'magma', 'RdBu_r', 'Greys_r', 'jet', 'rainbow', 'pink', 'bone', 'hsv', 'nipy_spectral'), value='RdBu_r'), Dropdown(description='Grid Filters', options=('tiltAngle', 'TMI'), value='tiltAngle'), ToggleButton(value=False, description='Refresh', icon='check', tooltip='Description'), ToggleButton(value=False, description='Export Grid', icon='check', tooltip='Description'), Output()), _dom_classes=('widget-interact',))